# Chicago Crime Data Analysis



Written by Bilal Fatian


This notebook forms a segment of the "Chicago Crime Data Analysis" project available on [my Github](https://github.com/bilalfatian/Chicago-Crime-Data-Analysis).

# Data Preprocessing
As of 5/3/2024, the [dataset](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2) provided by the city of Chicago on crime (excluding murders) contains over 8 millions rows and 22 columns. This [dataset](https://data.cityofchicago.org/Public-Safety/Homicides/k9xv-yxzs) contained the homicides, about 10,000, from the last 20 years. To facilitate early exploration of the data and focus on more recent, relevant trends, I removed crimes from before 2014, unneeded columns, and rows with nulls. The reduced dataset contained just under 3 million rows of crimes  
  
After reducing the size of the dataset, I cleaned up the text columns by manually matching values of each column with a smaller subset of categories in excel, mapped the Community Area ID's to their name and group (e.g. Community Area 8 maps to Near North Side and Central) based on [this](https://en.wikipedia.org/wiki/Community_areas_in_Chicago) Wikipedia page, and added some categorical columns based on the date of the crime.  

Also added in the Community Area population sizes from the [2010 Census](https://www.chicago.gov/content/dam/city/depts/zlup/Zoning_Main_Page/Publications/Census_2010_Community_Area_Profiles/Census_2010_and_2000_CA_Populations.pdf) to allow for an approximated Crimes/Homicides per Capita calculation. Unfortunately, this data isn't provided year over year and as of 5/3/2024, the 2024 Census isn't available, which is why I needed to use 2010 population sizes.

## Table of Contents
- [Dataset Description](#original_desc)  
- [Reading in the Original Datasets and Combining Them](#read_in)  
- [Reducing the Number of Crimes](#reduce)  
- [Removing Unneeded Columns](#unneeded_columns)
- [Cleaning Up Text Columns](#clean_up)
- [Pulling in Community Area Names and Regions](#community_areas)
- [Creating New Datetime Columns based on the Date of the Crime](#date_columns)
- [Dropping Nulls from the Dataset](#nulls)
- [Saving the Cleaned Dataset](#saving)

<a id="original_desc">  

## Dataset Description:
These are the original column descriptions from the City of Chicago 
[website](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2)  

| Column Name  | Column Description |  
| :-:    | :-: |  
| ID           | Unique identifier for the record |
| Case Number  | The Chicago Police Department Records Division Number |
| Date         | Date when the incident occurred (sometimes an estimate) |
| Block        | The partially redacted address where the incident occurred, placing it on the same block as the actual address |
| IUCR         | Illinois Uniform Crime Reporting code |
| Primary Type | The primary description of the IUCR code |
| Description  | The secondary description of the IUCR code, a subcategory of the primary description |
| Location Description | Description of the location where the incident occurred |
| Arrest | Indicates whether an arrest was made |
| Domestic | Indicates whether the incident was domestic-related as defined by the Illinois Domestic Violence act |
| Beat | Indicates the beat where the incident occurred. A beat is the smallest police geographic area. 3 to 5 beats make up a police sector, and 3 sectors make up a police district |
| District | Indicates the police district where the incident occurred |
| Ward | The ward (City Council district) where the incident occurred |
| Community Area | Indicates the community area where the incident occurred (Chicago has 77 community areas) |
| FBI Code | Indicates the crime classification as outlined in the FBI's National Incident-Based Reporting System (NIBRS) |
| X Coordinate | The x coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block |
| Y Coordinate | The y coordinate of the location where the incident occurred in State Plane Illinois East NAD 1983 projection. This location is shifted from the actual location for partial redaction but falls on the same block |
| Year | The year the incident occurred |
| Updated On | Date and time the record was last updated |
| Latitude | The latitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block |
| Longitude | The longitude of the location where the incident occurred. This location is shifted from the actual location for partial redaction but falls on the same block |
| Location | The location where the incident occurred in a format that allows for creation of maps and other geographic operations on this data portal. This location is shifted from the actual location for partial redaction but falls on the same block |

<a id="read_in">

## Reading in the Original Datasets and Combining Them

In [1]:
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta

import warnings
warnings.filterwarnings('ignore')

In [5]:
crimes_general = pd.read_csv("./Data/Crimes_-_2001_to_Present_20240504.csv", parse_dates=['Date'])
crimes_murders = pd.read_csv("./Data/Homicides.csv", parse_dates=['Date'])

In [6]:
#combining general crimes and homicides
crimes = pd.concat([crimes_general, crimes_murders], ignore_index=True)

#converting arrest column from boolean to int
crimes['Arrest'] = crimes['Arrest'].astype(int)

In [7]:
print("There were {:,d} General Crimes from {} to {}".format(crimes_general.shape[0],
                                                           crimes_general.Date.min(),
                                                           crimes_general.Date.max()))
print()
print("There were {:,d} Homicides from {} to {}".format(crimes_murders.shape[0],
                                                           crimes_murders.Date.min(),
                                                           crimes_murders.Date.max()))
print()
print("There were {:,d} Total Crimes from {} to {}".format(crimes.shape[0],
                                                           crimes.Date.min(),
                                                           crimes.Date.max()))

There were 8,052,078 General Crimes from 2001-01-01 00:00:00 to 2024-04-26 00:00:00

There were 13,174 Homicides from 2001-01-01 10:40:00 to 2024-04-23 20:23:00

There were 8,065,252 Total Crimes from 2001-01-01 00:00:00 to 2024-04-26 00:00:00


<a id="reduce">

## Reducing the Number of Crimes

Because of the large size of the original dataset, I am removing crimes from before 2014 to allow for relevant inferences to the present. I am also removing the crimes from 2024 so as to only include crimes that have had what I believe to be a reasonable amount of time to make an arrest.

In [8]:
#removing crimes from over 10 years ago and the most recent 4 months
crimes_cleaned = crimes[(crimes.Year >= 2014) & (crimes.Year <= 2024)].copy()

#displaying the results
print("There were {:,d} of Crimes from {} to {}".format(crimes_cleaned.shape[0],
                                                        crimes_cleaned.Date.min(),
                                                        crimes_cleaned.Date.max()))

There were 2,615,025 of Crimes from 2014-01-01 00:00:00 to 2024-04-26 00:00:00


<a id="unneeded_columns">

## Removing Unneeded Columns

In [9]:
#creating a list of the columns to drop
drop_cols = ['ID','Case Number', 'Block', 'Description', 
             'Beat', 'District', 'Ward', 'IUCR', 'FBI Code', 
             'Location', 'X Coordinate', 'Y Coordinate', 
             'Updated On']

#dropping the columns
crimes_cleaned.drop(labels=drop_cols, axis=1, inplace=True)

#displaying the results
crimes_cleaned.head()

,Date,Primary Type,Location Description,Arrest,Domestic,Community Area,Year,Latitude,Longitude
1,2021-05-24 15:06:00,HOMICIDE,STREET,1,False,19.0,2021,41.917838,-87.755969
2,2021-06-26 09:24:00,HOMICIDE,PARKING LOT,1,False,13.0,2021,41.995219,-87.713355
3,2023-11-09 07:30:00,BURGLARY,APARTMENT,0,False,5.0,2023,41.952345,-87.677975
4,2023-11-12 07:59:00,BATTERY,SMALL RETAIL STORE,1,False,44.0,2023,41.737751,-87.604856
6,2023-09-06 17:00:00,CRIMINAL DAMAGE,PARKING LOT / GARAGE (NON RESIDENTIAL),0,False,32.0,2023,41.886018,-87.633938


<a id="clean_up">

## Cleaning Up Text Columns

### Primary Type

In [10]:
#original values
crimes_cleaned['Primary Type'].unique()

array(['HOMICIDE', 'BURGLARY', 'BATTERY', 'CRIMINAL DAMAGE',
       'DECEPTIVE PRACTICE', 'CRIMINAL SEXUAL ASSAULT', 'THEFT',
       'OFFENSE INVOLVING CHILDREN', 'MOTOR VEHICLE THEFT', 'ROBBERY',
       'ASSAULT', 'SEX OFFENSE', 'OTHER OFFENSE', 'WEAPONS VIOLATION',
       'STALKING', 'CRIMINAL TRESPASS', 'PROSTITUTION', 'ARSON',
       'NARCOTICS', 'KIDNAPPING', 'CONCEALED CARRY LICENSE VIOLATION',
       'INTERFERENCE WITH PUBLIC OFFICER', 'PUBLIC PEACE VIOLATION',
       'OBSCENITY', 'LIQUOR LAW VIOLATION', 'INTIMIDATION', 'GAMBLING',
       'HUMAN TRAFFICKING', 'OTHER NARCOTIC VIOLATION', 'NON-CRIMINAL',
       'PUBLIC INDECENCY', 'CRIM SEXUAL ASSAULT', 'NON - CRIMINAL',
       'NON-CRIMINAL (SUBJECT SPECIFIED)', 'RITUALISM'], dtype=object)

In [11]:
#list of other values to group under "NON-CRIMINAL"
non_criminal_list = ['NON - CRIMINAL','NON-CRIMINAL (SUBJECT SPECIFIED)']

#replacing the other values with "NON-CRIMINAL"
crimes_cleaned['Primary Type'].replace(to_replace = non_criminal_list,
                                       value='NON-CRIMINAL', regex=False,
                                       inplace=True)

#Converting values to title case
crimes_cleaned['Primary Type'] = crimes_cleaned['Primary Type'].str.title()

#displaying the results
crimes_cleaned['Primary Type'].unique()

array(['Homicide', 'Burglary', 'Battery', 'Criminal Damage',
       'Deceptive Practice', 'Criminal Sexual Assault', 'Theft',
       'Offense Involving Children', 'Motor Vehicle Theft', 'Robbery',
       'Assault', 'Sex Offense', 'Other Offense', 'Weapons Violation',
       'Stalking', 'Criminal Trespass', 'Prostitution', 'Arson',
       'Narcotics', 'Kidnapping', 'Concealed Carry License Violation',
       'Interference With Public Officer', 'Public Peace Violation',
       'Obscenity', 'Liquor Law Violation', 'Intimidation', 'Gambling',
       'Human Trafficking', 'Other Narcotic Violation', 'Non-Criminal',
       'Public Indecency', 'Crim Sexual Assault', 'Ritualism'],
      dtype=object)

### Location Description

In [12]:
#original values of Location Description
crimes_cleaned['Location Description'].unique()

array(['STREET', 'PARKING LOT', 'APARTMENT', 'SMALL RETAIL STORE',
       'PARKING LOT / GARAGE (NON RESIDENTIAL)', 'RESIDENCE - GARAGE',
       'RESIDENCE - PORCH / HALLWAY', 'CURRENCY EXCHANGE', 'RESIDENCE',
       'AUTO / BOAT / RV DEALERSHIP',
       'POLICE FACILITY / VEHICLE PARKING LOT', 'DEPARTMENT STORE',
       'CHA PARKING LOT / GROUNDS', 'RESTAURANT', 'GROCERY FOOD STORE',
       'APPLIANCE STORE', 'CAR WASH', 'HOSPITAL BUILDING / GROUNDS',
       'COMMERCIAL / BUSINESS OFFICE', 'SIDEWALK',
       'DRIVEWAY - RESIDENTIAL', 'OTHER (SPECIFY)', 'PARK PROPERTY',
       'ALLEY', 'BANK', 'DRUG STORE', 'VEHICLE NON-COMMERCIAL',
       'RESIDENCE - YARD (FRONT / BACK)', 'BAR OR TAVERN',
       'LAKEFRONT / WATERFRONT / RIVERBANK', 'SCHOOL - PUBLIC BUILDING',
       'AIRPORT TERMINAL LOWER LEVEL - SECURE AREA',
       'NURSING / RETIREMENT HOME', 'HOTEL / MOTEL', 'CONVENIENCE STORE',
       'GAS STATION', 'CTA BUS STOP',
       'AIRPORT TERMINAL UPPER LEVEL - NON-SECURE AREA',
     

In [13]:
#reading in manual mapping of unique original values to new ones
location = pd.read_csv('Data/Location.csv')
location.head()

,Original Value,New Value
0,RESIDENCE,Residence
1,OTHER,Other
2,HOTEL/MOTEL,Hotel/Motel
3,APARTMENT,Residence
4,SIDEWALK,Sidewalk


In [14]:
#merging the crimes_cleaned df with the location df 
crimes_cleaned = crimes_cleaned.merge(location, how='left', 
                                      left_on='Location Description',
                                      right_on='Original Value')

#removing the original value columns
crimes_cleaned.drop(labels=['Location Description','Original Value'],axis=1,
                    inplace=True)

#renaming the new value column as Location Description
crimes_cleaned.rename(columns={'New Value':'Location Description'}, inplace=True)

#displaying the results
crimes_cleaned.head()

,Date,Primary Type,Arrest,Domestic,Community Area,Year,Latitude,Longitude,Location Description
0,2021-05-24 15:06:00,Homicide,1,False,19.0,2021,41.917838,-87.755969,Street
1,2021-06-26 09:24:00,Homicide,1,False,13.0,2021,41.995219,-87.713355,Parking Lot
2,2023-11-09 07:30:00,Burglary,0,False,5.0,2023,41.952345,-87.677975,Residence
3,2023-11-12 07:59:00,Battery,1,False,44.0,2023,41.737751,-87.604856,Store
4,2023-09-06 17:00:00,Criminal Damage,0,False,32.0,2023,41.886018,-87.633938,NaN


<a id="community_areas">

## Pulling in Community Area Names and Regions

In [15]:
#reading in the mapping of the Community Area ID to the Name and Region
#values from Wikipedia
community_areas = pd.read_csv('Data/Community_Areas.csv')
community_areas.head()

,ID,Name,Region,2010 Population
0,8,Near North Side,Central,80484
1,32,Loop,Central,29283
2,33,Near South Side,Central,21390
3,5,North Center,North Side,31867
4,6,Lake View,North Side,94368


In [16]:
#merging the community area df with the reduced crimes df
crimes_cleaned = crimes_cleaned.merge(community_areas, how='left', 
                                      left_on='Community Area',
                                      right_on='ID')

#dropping the ID columns
crimes_cleaned.drop(labels=['Community Area','ID'],axis=1,inplace=True)

#renaming the Name column as Community Area
crimes_cleaned.rename(columns={'Name':'Community Area'}, inplace=True)

#displaying the results
crimes_cleaned.head()

,Date,Primary Type,Arrest,Domestic,Year,Latitude,Longitude,Location Description,Community Area,Region,2010 Population
0,2021-05-24 15:06:00,Homicide,1,False,2021,41.917838,-87.755969,Street,Belmont Cragin,Northwest Side,78743.0
1,2021-06-26 09:24:00,Homicide,1,False,2021,41.995219,-87.713355,Parking Lot,North Park,Far North Side,17931.0
2,2023-11-09 07:30:00,Burglary,0,False,2023,41.952345,-87.677975,Residence,North Center,North Side,31867.0
3,2023-11-12 07:59:00,Battery,1,False,2023,41.737751,-87.604856,Store,Chatham,Far Southeast Side,31028.0
4,2023-09-06 17:00:00,Criminal Damage,0,False,2023,41.886018,-87.633938,NaN,Loop,Central,29283.0


<a id="date_columns">

## Creating New Datetime Columns based on the Date of the Crime

In [17]:
#adding a column for the month of the crime
crimes_cleaned['Month'] = crimes_cleaned.Date.dt.month

#adding a column for the day of week of the crime
crimes_cleaned['Day of Week'] = crimes_cleaned.Date.dt.day_name()

#creating index vars for time of day (morning, afternoon, evening, and night)
morning_idx   = (crimes_cleaned.Date.dt.time >= dt.time( 5)) & (crimes_cleaned.Date.dt.time < dt.time(12)) # 5am to 12pm
afternoon_idx = (crimes_cleaned.Date.dt.time >= dt.time(10)) & (crimes_cleaned.Date.dt.time < dt.time(17)) #12pm to  5pm
evening_idx   = (crimes_cleaned.Date.dt.time >= dt.time(17)) & (crimes_cleaned.Date.dt.time < dt.time(20)) # 5pm to  8pm
night_idx     = (crimes_cleaned.Date.dt.time >= dt.time(20)) | (crimes_cleaned.Date.dt.time < dt.time( 5)) # 8pm to  5am

#adding a column for the time of day of the crime
crimes_cleaned['Time of Day'] = ""
crimes_cleaned.loc[morning_idx,   'Time of Day'] = "Morning"
crimes_cleaned.loc[afternoon_idx, 'Time of Day'] = "Afternoon"
crimes_cleaned.loc[evening_idx,   'Time of Day'] = "Evening"
crimes_cleaned.loc[night_idx,     'Time of Day'] = "Night"

#creating index vars for season of year (spring, summer, fall, winter)
spring_idx = (crimes_cleaned.Month >=  3) & (crimes_cleaned.Month <  6)
summer_idx = (crimes_cleaned.Month >=  6) & (crimes_cleaned.Month <  9)
fall_idx   = (crimes_cleaned.Month >=  9) & (crimes_cleaned.Month < 12)
winter_idx = (crimes_cleaned.Month >= 12) | (crimes_cleaned.Month <  3)

#adding a column for the season the crime occurred in
crimes_cleaned['Season'] = ""
crimes_cleaned.loc[spring_idx, 'Season'] = "Spring"
crimes_cleaned.loc[summer_idx, 'Season'] = "Summer"
crimes_cleaned.loc[fall_idx,   'Season'] = "Fall"
crimes_cleaned.loc[winter_idx, 'Season'] = "Winter"

In [18]:
crimes_cleaned.head()

,Date,Primary Type,Arrest,Domestic,Year,Latitude,Longitude,Location Description,Community Area,Region,2010 Population,Month,Day of Week,Time of Day,Season
0,2021-05-24 15:06:00,Homicide,1,False,2021,41.917838,-87.755969,Street,Belmont Cragin,Northwest Side,78743.0,5,Monday,Afternoon,Spring
1,2021-06-26 09:24:00,Homicide,1,False,2021,41.995219,-87.713355,Parking Lot,North Park,Far North Side,17931.0,6,Saturday,Morning,Summer
2,2023-11-09 07:30:00,Burglary,0,False,2023,41.952345,-87.677975,Residence,North Center,North Side,31867.0,11,Thursday,Morning,Fall
3,2023-11-12 07:59:00,Battery,1,False,2023,41.737751,-87.604856,Store,Chatham,Far Southeast Side,31028.0,11,Sunday,Morning,Fall
4,2023-09-06 17:00:00,Criminal Damage,0,False,2023,41.886018,-87.633938,NaN,Loop,Central,29283.0,9,Wednesday,Evening,Fall


<a id="saving">

<a id="nulls">

## Dropping Nulls from the Dataset
Because of the relatively small number of rows with null values, I'm going to drop all rows with nulls.

In [19]:
print("Number of Rows: {:,d}".format(crimes_cleaned.shape[0]))
print()
for col in crimes_cleaned.columns:
    print("{} has {:,d} nulls".format(col,crimes_cleaned[col].isna().sum()))

Number of Rows: 2,615,025

Date has 0 nulls
Primary Type has 0 nulls
Arrest has 0 nulls
Domestic has 0 nulls
Year has 0 nulls
Latitude has 38,952 nulls
Longitude has 38,952 nulls
Location Description has 144,466 nulls
Community Area has 2 nulls
Region has 2 nulls
2010 Population has 2 nulls
Month has 0 nulls
Day of Week has 0 nulls
Time of Day has 0 nulls
Season has 0 nulls


In [20]:
crimes_cleaned.dropna(inplace=True,axis=0)

print("Number of Rows: {:,d}".format(crimes_cleaned.shape[0]))
print()
for col in crimes_cleaned.columns:
    print("{} has {:,d} nulls".format(col,crimes_cleaned[col].isna().sum()))

Number of Rows: 2,439,089

Date has 0 nulls
Primary Type has 0 nulls
Arrest has 0 nulls
Domestic has 0 nulls
Year has 0 nulls
Latitude has 0 nulls
Longitude has 0 nulls
Location Description has 0 nulls
Community Area has 0 nulls
Region has 0 nulls
2010 Population has 0 nulls
Month has 0 nulls
Day of Week has 0 nulls
Time of Day has 0 nulls
Season has 0 nulls


## Saving the Cleaned Dataset

In [21]:
crimes_cleaned.to_csv("Data/crimes_cleaned.csv", index=False)